In [28]:
import pandas as pd 
import io


# EMERGENCIA HABITACIONAL EN VALENCIA


***

### TEMA

![EDA.png](./img/portada_presentacion.jpg)

Se abordará la crisis habitacional que afecta a la comunidad valenciana desde diferentes flancos.

### HIPÓTESIS

La dificultad para alquiler un piso en valencia está relacionada con:
- La disminucion de la oferta de pisos en alquiler,
- El aumento de la demanda por los saldos migratorios en la comunidad valenciana,
- Tasas hipotecarias prohibitivas para acceder a la vivienda propia
- Aumento de los valores de alquiler 




## OBTENCIÓN DE LOS DATOS

### DATASETS Y FUENTES ALTERNATIVAS DE DATOS

[REPORTE SALA DE PRENSA IDEALISTA ](Https://www.idealista.com/sala-de-prensa/informes-precio-vivienda/alquiler/comunitat-valenciana/report/) <br>

[NOTA DE PRENSA INE ](https://ine.es/prensa/cp_e2022_p.pdf) <br>

[EUROPA PRESS ](https://www.epdata.es/datos/poblacion-inmigrantes-emigrantes-otros-datos-habitantes-cada-comunidad-autonoma/2/comunidad-valenciana/299)<br>

[EVOLUCUIN EURIBOR ](https://www.epdata.es/variacion-mensual-euribor-ultimos-24-meses/0ac73426-79a0-40c1-873f-44befbd9a576)

También incluye el origen de estos datos y su fuente.

In [ ]:
# describe la fuente de tus datos e incluye los links en Markdown

Muestra mediante un head() los principales datasets con los que vas a trabajar

In [16]:
ejecuciones_hipotecarias = pd.read_csv(io.open("./raw/ejecuciones_hipotecarias_comunidad_valenciana.csv", encoding='latin-1'), sep="\t")
ejecuciones_hipotecarias.head()

,Total Nacional,Comunidades y Ciudades Autónomas,Titular de la vivienda,Periodo,Total
0,Total,10 Comunitat Valenciana,Persona física,2022T2,817.0
1,Total,10 Comunitat Valenciana,Persona física,2022T1,747.0
2,Total,10 Comunitat Valenciana,Persona física,2021T4,793.0
3,Total,10 Comunitat Valenciana,Persona física,2021T3,590.0
4,Total,10 Comunitat Valenciana,Persona física,2021T2,762.0


In [25]:
tipos_interes = pd.read_csv(io.open("./raw/interes fijo vs variable.csv", encoding='latin-1'), sep="\t")
tipos_interes.head()

,Tipo de interés,Naturaleza de la finca,Periodo,Total
0,Fijo,Viviendas,2022M07,"75,4"
1,Fijo,Viviendas,2022M06,"73,0"
2,Fijo,Viviendas,2022M05,"72,8"
3,Fijo,Viviendas,2022M04,"75,3"
4,Fijo,Viviendas,2022M03,"72,7"


In [17]:
IPC_Vivienda = pd.read_csv(io.open("./raw/Índice de Precios de Vivienda CV MAD CAT.csv", encoding='latin-1'), sep="\t")
IPC_Vivienda.head()

,Total Nacional,Comunidades y Ciudades Autónomas,"General, vivienda nueva y de segunda mano",Índices y tasas,Periodo,Total
0,Nacional,09 Cataluña,Vivienda segunda mano,Variación anual,2022T2,"7,2"
1,Nacional,09 Cataluña,Vivienda segunda mano,Variación anual,2022T1,"7,5"
2,Nacional,09 Cataluña,Vivienda segunda mano,Variación anual,2021T4,"6,3"
3,Nacional,09 Cataluña,Vivienda segunda mano,Variación anual,2021T3,"3,5"
4,Nacional,09 Cataluña,Vivienda segunda mano,Variación anual,2021T2,"2,9"


In [18]:
saldo_migratorio = pd.read_csv(io.open("./raw/saldo migratorio interno.csv", encoding='latin-1'), sep="\t")
saldo_migratorio.head()

,Comunidades y ciudades autónomas,Sexo,Edad,Periodo,Total
0,01 Andalucía,Ambos sexos,Total,2021S2,284.000
1,01 Andalucía,Ambos sexos,Total,2021S1,4.365
2,01 Andalucía,Ambos sexos,Total,2020S2,3.418
3,01 Andalucía,Ambos sexos,Total,2020S1,691.000
4,01 Andalucía,Ambos sexos,Total,2019S2,-1.356


In [24]:
migraciones_exteriores = pd.read_csv(io.open("./raw/migraciones exteriores.csv", encoding='latin-1'), sep="\t")
migraciones_exteriores.head()

,ï»¿Provincias,Sexo,Nacionalidad,Grupo quinquenal de edad,Periodo,Total
0,Total Nacional,Ambos sexos,Total,Total,2021,148.677
1,Total Nacional,Ambos sexos,Total,Total,2020,219.357
2,Total Nacional,Ambos sexos,Total,Total,2019,454.232
3,Total Nacional,Ambos sexos,Total,Total,2018,334.158
4,Total Nacional,Ambos sexos,Total,Total,2017,163.272


Trabajando en la extraccion del SOUP

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())

url = "https://www.idealista.com/sala-de-prensa/informes-precio-vivienda/alquiler/comunitat-valenciana/report/"
driver.get(url)

Cookies = driver.find_element(by= By.XPATH,value="/html/body/div[1]/div/div/div/div/div[2]/button[2]")
Cookies.click()

soup = bs(driver.page_source,"lxml")

soup2 = soup.select("td.table__cell")
soup2
soup2[0:7]
soup2[7:14]
soup2[14:21]

d = {}
for i in range(int(len(soup2)/7)):
    d.setdefault("Localidad",  []).append(soup2[i*7-7].get_text())
    d.setdefault("Precio_(€/m2)" , []).append(soup2[i*7-6].get_text().split()[0])
    d.setdefault("Variacion_trimestral(%)", []).append(eval(soup2[i*7-4].get_text().split()[0] + soup2[i*7-4].get_text().split()[1].replace(",", ".")))
    d.setdefault("Variacion_anual(%)",      []).append(eval(soup2[i*7-3].get_text().split()[0] + soup2[i*7-3].get_text().split()[1].replace(",", ".")))
    d.setdefault("Maximo_historico(€/m2)",     []).append(soup2[i*7-2].get_text().split()[0])
    d.setdefault("Variacion_Maximo(%)",     []).append(eval(soup2[i*7-3].get_text().split()[0] + soup2[i*7-3].get_text().split()[1].replace(",", ".")))
    

TABLA_PROP_INTERANUAL = pd.DataFrame(d)
TABLA_PROP_INTERANUAL

[<td class="table__cell" data-sortable="Comunitat Valenciana">Comunitat Valenciana</td>,
 <td class="table__cell" data-sortable="8.93495">8,9 €/m2</td>,
 <td class="table__cell table__cell--red" data-sortable="-0.702477">- 0,7 %</td>,
 <td class="table__cell table__cell--green" data-sortable="4.75313">+ 4,8 %</td>,
 <td class="table__cell table__cell--green" data-sortable="15.849">+ 15,8 %</td>,
 <td class="table__cell" data-sortable="9.39699">9,4 €/m2 feb 2006</td>,
 <td class="table__cell table__cell--red" data-sortable="-4.9168936010361">- 4,9 %</td>]

[<td class="table__cell" data-sortable="Comunitat Valenciana">Comunitat Valenciana</td>,
 <td class="table__cell" data-sortable="8.93495">8,9 €/m2</td>,
 <td class="table__cell table__cell--red" data-sortable="-0.702477">- 0,7 %</td>,
 <td class="table__cell table__cell--green" data-sortable="4.75313">+ 4,8 %</td>,
 <td class="table__cell table__cell--green" data-sortable="15.849">+ 15,8 %</td>,
 <td class="table__cell" data-sortable="9.39699">9,4 €/m2 feb 2006</td>,
 <td class="table__cell table__cell--red" data-sortable="-4.9168936010361">- 4,9 %</td>]